In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install wandb -qqq

In [3]:
from wandb.keras import WandbCallback
import wandb

In [4]:
# Configure the sweep
sweep_config = {
    'name' : 'Neuron & LR Sweep - Batch Size 128',
    'method': 'grid',
    'parameters': {
        'learning_rate': {
            'values': [5e-4, 1e-5]
        },
        'fc1_num_neurons': {
            'values': [512, 1024]
        },
        'fc3_num_neurons': {
            'values': [256, 512]
        },
    }
}

In [5]:
WANDB_API_KEY = '002b3552fd78e3559e38c7317ef0d9eefd0c7c6d'
sweep_id = wandb.sweep(sweep_config, project='nn_project_tuning')

Create sweep with ID: j0r78xbh
Sweep URL: https://wandb.ai/burntice/nn_project_tuning/sweeps/j0r78xbh


In [6]:
# sweep reference: 
# https://colab.research.google.com/drive/1gKixa6hNUB8qrn1CfHirOfTEQm0qLCSS?usp=sharing

In [7]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import pandas as pd
import tensorflow as tf


# Load dataset as dataframe
base_path = '/content/drive/My Drive/NN Project/'
df = pd.read_csv(base_path + 'aligned_gender.txt', sep='\t')
df['datadir'] = base_path + df['datadir'].astype(str)

# Train test split
train_val_df, test_df = train_test_split(df, test_size=0.2)
train_df, val_df = train_test_split(train_val_df, test_size=0.2)

# function to call for hyperparameter sweep
def train():
    # Fix seed
    seed = 7
    np.random.seed(seed)
    tf.random.set_seed(seed)

    # default hyperparameters
    defaults = {
        'epochs': 20,
        'batch_size': 128,
        'learning_rate': 1e-5,
        'weight_decay': 0.01,
        'fc1_num_neurons': 1024,
        'fc2_num_neurons': 512,
        'fc3_num_neurons': 256,
        'dropout1_rate': 0,
        'dropout2_rate': 0,
        'dropout3_rate': 0,
        'seed': seed,
        'optimizer': 'adam',
        'hidden_activation': 'relu',
        'output_activation': 'sigmoid',
        'loss_function': 'binary_crossentropy',
        'metrics': ['accuracy'],
    }

    wandb.init(
        config=defaults,
        name='Hyperparameter Tuning',
        project='nn_project',
    )
    config = wandb.config

    # Define model
    mobile_net_v2 = tf.keras.applications.MobileNetV2(
        include_top=False,
        pooling='avg',
        weights='imagenet',
        input_shape=(224,224,3),
    )

    fc1 = tf.keras.layers.Dense(
        config.fc1_num_neurons,
        activation=config.hidden_activation,
        kernel_regularizer=tf.keras.regularizers.L2(l2=config.weight_decay),
    )

    fc2 = tf.keras.layers.Dense(
        config.fc2_num_neurons,
        activation=config.hidden_activation,
        kernel_regularizer=tf.keras.regularizers.L2(l2=config.weight_decay),
    )

    fc3 = tf.keras.layers.Dense(
        config.fc3_num_neurons,
        activation=config.hidden_activation,
        kernel_regularizer=tf.keras.regularizers.L2(l2=config.weight_decay),
    )

    bn1 = tf.keras.layers.BatchNormalization()
    bn2 = tf.keras.layers.BatchNormalization()
    bn3 = tf.keras.layers.BatchNormalization()
    bn4 = tf.keras.layers.BatchNormalization()

    dropout1 = tf.keras.layers.Dropout(rate=config.dropout1_rate)
    dropout2 = tf.keras.layers.Dropout(rate=config.dropout2_rate)
    dropout3 = tf.keras.layers.Dropout(rate=config.dropout3_rate)

    model = tf.keras.models.Sequential([
        mobile_net_v2,
        tf.keras.layers.Flatten(),
        bn1,
        fc1,
        dropout1,
        bn2,
        fc2,
        dropout2,
        bn3,
        fc3,
        dropout3,
        bn4,
        tf.keras.layers.Dense(1, activation=config.output_activation),
    ])

    # Compile model 
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=config.learning_rate),
        loss=config.loss_function,
        metrics=config.metrics,
    )

    # Load images into keras image generator 
    datagen_train = ImageDataGenerator(
        preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input,
    )
    datagen_val = ImageDataGenerator(
        preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input,
    )
    datagen_test = ImageDataGenerator(
        preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input,
    )

    train_generator = datagen_train.flow_from_dataframe(
        dataframe=train_df,
        x_col='datadir',
        y_col='gender',
        batch_size=config.batch_size,
        seed=seed,
        shuffle=True,
        class_mode='raw',
        target_size=(224,224),
    )

    val_generator = datagen_train.flow_from_dataframe(
        dataframe=val_df,
        x_col='datadir',
        y_col='gender',
        batch_size=config.batch_size,
        seed=seed,
        shuffle=True,
        class_mode='raw',
        target_size=(224,224),
    )

    test_generator = datagen_test.flow_from_dataframe(
        dataframe=test_df,
        x_col='datadir',
        y_col='gender',
        batch_size=config.batch_size,
        seed=seed,
        shuffle=True,
        class_mode='raw',
        target_size=(224,224),
    )

    model.fit(
        train_generator,
        validation_data=val_generator,
        epochs=config.epochs,
        callbacks=[WandbCallback()],
    )

In [8]:
wandb.agent(sweep_id, train)

wandb: Agent Starting Run: y0cx5z2j with config:
wandb: 	fc1_num_neurons: 512
wandb: 	fc3_num_neurons: 256
wandb: 	learning_rate: 0.0005
wandb: Currently logged in as: burntice (use `wandb login --relogin` to force relogin)
wandb: WARNING Ignored wandb.init() arg project when running a sweep


9412608/9406464 [==============================] - 0s 0us/step
Found 7804 validated image filenames.
Found 1951 validated image filenames.
Found 2439 validated image filenames.
Epoch 1/20
61/61 [==============================] - 3386s 56s/step - loss: 13.4429 - accuracy: 0.8301 - val_loss: 11.0394 - val_accuracy: 0.5720
Epoch 2/20
61/61 [==============================] - 150s 2s/step - loss: 7.7896 - accuracy: 0.9527 - val_loss: 5.9759 - val_accuracy: 0.7447
Epoch 3/20
61/61 [==============================] - 149s 2s/step - loss: 4.0595 - accuracy: 0.9768 - val_loss: 4.0355 - val_accuracy: 0.5992
Epoch 4/20
61/61 [==============================] - 148s 2s/step - loss: 2.0179 - accuracy: 0.9786 - val_loss: 1.8330 - val_accuracy: 0.8247
Epoch 5/20
61/61 [==============================] - 148s 2s/step - loss: 0.9953 - accuracy: 0.9796 - val_loss: 2.4171 - val_accuracy: 0.6228
Epoch 6/20
61/61 [==============================] - 149s 2s/step - loss: 0.4795 - accuracy: 0.9830 - val_loss: 2.0

epoch,19
loss,0.04847
accuracy,0.98924
val_loss,2.29476
val_accuracy,0.49616
_step,19
_runtime,6376
_timestamp,1605959379
best_val_loss,0.4956
best_epoch,10


epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
loss,█▅▃▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
accuracy,▁▆▇▇████████████████
val_loss,█▅▃▂▂▂▃▂▁▁▁▁▂▂▂▂▁▂▃▂
val_accuracy,▂▅▃▇▃▂▄▄▇█▆▄▆▅▇▃▁▃▁▁
_step,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
_runtime,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
_timestamp,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██


wandb: Agent Starting Run: xp7fp6cq with config:
wandb: 	fc1_num_neurons: 512
wandb: 	fc3_num_neurons: 256
wandb: 	learning_rate: 1e-05
wandb: WARNING Ignored wandb.init() arg project when running a sweep


...

wandb: Agent Starting Run: s5qx5lfu with config:
wandb: 	fc1_num_neurons: 1024
wandb: 	fc3_num_neurons: 512
wandb: 	learning_rate: 1e-05
wandb: WARNING Ignored wandb.init() arg project when running a sweep


Found 7804 validated image filenames.
Found 1951 validated image filenames.
Found 2439 validated image filenames.
Epoch 1/20
61/61 [==============================] - 152s 2s/step - loss: 23.9544 - accuracy: 0.6121 - val_loss: 23.8233 - val_accuracy: 0.6094
Epoch 2/20
61/61 [==============================] - 150s 2s/step - loss: 23.5499 - accuracy: 0.7622 - val_loss: 23.5683 - val_accuracy: 0.6776
Epoch 3/20
61/61 [==============================] - 151s 2s/step - loss: 23.2396 - accuracy: 0.8332 - val_loss: 23.3212 - val_accuracy: 0.7078
Epoch 4/20
61/61 [==============================] - 150s 2s/step - loss: 22.9422 - accuracy: 0.8817 - val_loss: 23.0904 - val_accuracy: 0.7145
Epoch 5/20
61/61 [==============================] - 149s 2s/step - loss: 22.6554 - accuracy: 0.9179 - val_loss: 22.8521 - val_accuracy: 0.7258
Epoch 6/20
61/61 [==============================] - 149s 2s/step - loss: 22.3776 - accuracy: 0.9381 - val_loss: 22.6065 - val_accuracy: 0.7371
Epoch 7/20
61/61 [==========

epoch,19
loss,18.16883
accuracy,0.99923
val_loss,18.31238
val_accuracy,0.86366
_step,19
_runtime,3055
_timestamp,1605980696
best_val_loss,18.31238
best_epoch,19


epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
loss,██▇▇▆▆▆▅▅▅▄▄▄▃▃▃▂▂▁▁
accuracy,▁▄▅▆▇▇▇▇████████████
val_loss,██▇▇▇▆▆▆▅▅▅▄▄▃▃▃▂▂▁▁
val_accuracy,▁▃▄▄▄▅▅▅▅▆▆▆▇▇▇▇████
_step,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
_runtime,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
_timestamp,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.
